# Encoder-decoder architecture

Encoder-decoder architectures are about converting anything to anything, including
 * Machine translation and spoken dialogue systems
 * [Image captioning](http://mscoco.org/dataset/#captions-challenge2015) and [image2latex](https://openai.com/requests-for-research/#im2latex) (convolutional encoder, recurrent decoder)
 * Generating [images by captions](https://arxiv.org/abs/1511.02793) (recurrent encoder, convolutional decoder)
 * Grapheme2phoneme - convert words to transcripts

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE
import numpy as np
import matplotlib.pyplot as plt
from vocab import Vocab
from sklearn.model_selection import train_test_split
from my_models import BasicModel
import torch
from IPython.display import clear_output
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from tqdm.notebook import tqdm

# Our task: machine translation

We gonna try our encoder-decoder models on russian to english machine translation problem. More specifically, we'll translate hotel and hostel descriptions.

* Data will be tokenized with WordPunctTokenizer.

* Our data lines contain unique rare words. If we operate on a word level, we will have to deal with large vocabulary size. If instead we use character-level models, it would take lots of iterations to process a sequence. This time we're gonna pick something inbetween.

* One popular approach is called [Byte Pair Encoding](https://github.com/rsennrich/subword-nmt) aka __BPE__. The algorithm starts with a character-level tokenization and then iteratively merges most frequent pairs for N iterations. This results in frequent words being merged into a single token and rare words split into syllables or even characters.



In [ ]:
!pip3 install torch>=1.3.0
!pip3 install subword-nmt &> log
# !wget https://www.dropbox.com/s/yy2zqh34dyhv07i/data.txt?dl=1 -O data.txt
# !wget https://raw.githubusercontent.com/yandexdataschool/nlp_course/2020/week04_seq2seq/vocab.py -O vocab.py
# thanks to tilda and deephack teams for the data, Dmitry Emelyanenko for the code :)

In [ ]:
# def tokenize(x):
#     return ' '.join(nltk_tokenizer.tokenize(x.lower()))


# nltk_tokenizer = WordPunctTokenizer()

# # split and tokenize the data
# with open('train.en', 'w') as f_src,  open('train.ru', 'w') as f_dst:
#     for line in open('data.txt'):
#         src_line, dst_line = line.strip().split('\t')
#         f_src.write(tokenize(src_line) + '\n')
#         f_dst.write(tokenize(dst_line) + '\n')

# # build and apply bpe vocs
# bpe = {}
# for lang in ['en', 'ru']:
#     learn_bpe(open('./train.' + lang), open('bpe_rules.' + lang, 'w'), num_symbols=8000)
#     bpe[lang] = BPE(open('./bpe_rules.' + lang))
    
#     with open('train.bpe.' + lang, 'w') as f_out:
#         for line in open('train.' + lang):
#             f_out.write(bpe[lang].process_line(line.strip()) + '\n')

## Building vocabularies

We now need to build vocabularies that map strings to token ids and vice versa. We're gonna need these fellas when we feed training data into model or convert output matrices into words.

## Let's take a look at some sample data

In [ ]:
data_inp = np.array(open('./train.bpe.ru').read().split('\n'))
data_out = np.array(open('./train.bpe.en').read().split('\n'))

assert len(data_inp) == len(data_out), 'Number of inp & out sequences must be the same'

print('Number of sequences: %s' % len(data_inp), end='\n\n')

train_inp, dev_inp, train_out, dev_out = train_test_split(data_inp, data_out, test_size=0.05,
                                                          random_state=0)
for i in range(3):
    print('Inp:', train_inp[i])
    print('Out:', train_out[i], end='\n\n')

In [ ]:
inp_voc = Vocab.from_lines(train_inp)
out_voc = Vocab.from_lines(train_out)

In [ ]:
# Here's how you cast lines into ids and backwards
batch_lines = sorted(train_inp, key=len)[5:10]
batch_ids = inp_voc.to_matrix(batch_lines)
batch_lines_restored = inp_voc.to_lines(batch_ids)

print('Lines: %s' % batch_lines, end='\n\n')
print('Words to ids (0 = BOS, 1 = EOS):\n %s' % batch_ids, end='\n\n') # BOS = Begin Of Sentence
print('Back to words: %s' % batch_lines_restored)

## Draw source and translation length distributions to estimate the scope of the task

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.title("source length")
plt.hist(list(map(len, map(str.split, train_inp))), bins=20);

plt.subplot(1, 2, 2)
plt.title("translation length")
plt.hist(list(map(len, map(str.split, train_out))), bins=20);

## Encoder-decoder model

The code below contains a template for a simple encoder-decoder model: single GRU encoder/decoder, no attention or anything

In [ ]:
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
device

## Debugging area

In [ ]:
model = BasicModel(inp_voc, out_voc).to(device)

dummy_inp_tokens = inp_voc.to_matrix(sorted(train_inp, key=len)[5:10]).to(device)
dummy_out_tokens = out_voc.to_matrix(sorted(train_out, key=len)[5:10]).to(device)

h0 = model.encode(dummy_inp_tokens)
h1, logits1 = model.decode_step(h0, torch.arange(len(dummy_inp_tokens), device=device))

assert isinstance(h1, list) and len(h1) == len(h0)
assert h1[0].shape == h0[0].shape and not torch.allclose(h1[0], h0[0])
assert logits1.shape == (len(dummy_inp_tokens), len(out_voc))

logits_seq = model.decode(h0, dummy_out_tokens)
assert logits_seq.shape == (dummy_out_tokens.shape[0], dummy_out_tokens.shape[1], len(out_voc))

# full forward
logits_seq2 = model(dummy_inp_tokens, dummy_out_tokens)
assert logits_seq2.shape == logits_seq.shape

dummy_translations, dummy_states = model.translate_lines(train_inp[:2], device, max_len=25)
translations = '\n'.join([line for line in dummy_translations])
print('Translations without training:\n%s' % translations)

## Training loss

Our training objective is almost the same as it was for neural language models:
$$ L = {\frac1{|D|}} \sum_{X, Y \in D} \sum_{y_t \in Y} - \log p(y_t \mid y_1, \dots, y_{t-1}, X, \theta) $$

where $|D|$ is the __total length of all sequences__, including BOS and first EOS, but excluding PAD.

In [ ]:
from commons import compute_loss

In [ ]:
dummy_loss = compute_loss(model, dummy_inp_tokens, dummy_out_tokens)
print('Loss: %s' % dummy_loss)
assert np.allclose(dummy_loss.item(), 7.5, rtol=0.1, atol=0.1), 'Sorry for your loss'

# test autograd
dummy_loss.backward()
for name, param in model.named_parameters():
    assert param.grad is not None and abs(param.grad.max()) != 0, 'Param %s received no gradients' % name

## Evaluation: BLEU

Machine translation is commonly evaluated with [BLEU](https://en.wikipedia.org/wiki/BLEU) score. This metric simply computes which fraction of predicted n-grams is actually present in the reference translation. It does so for n=1,2,3 and 4 and computes the geometric average with penalty if translation is shorter than reference.

While BLEU [has many drawbacks](http://www.cs.jhu.edu/~ccb/publications/re-evaluating-the-role-of-bleu-in-mt-research.pdf), it still remains the most commonly used metric and one of the simplest to compute.

In [ ]:
from commons import compute_bleu

In [ ]:
# compute_bleu(model, dev_inp, dev_out)

## Training loop

Training encoder-decoder models isn't that different from any other models: sample batches, compute loss, backprop and update

In [ ]:
from tqdm import tqdm

def train_model(model, n_iters=15000, batch_size=64, lr=1e-3, n_iters_break=500):
    metrics = {'train_loss': [], 'dev_bleu': [] }
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    
    best_weights = model.state_dict()
    best_bleu = -1
    best_step = -1
    with tqdm(range(n_iters)) as progress_bar:
        for i in progress_bar:
            step = len(metrics['train_loss']) + 1
            batch_ix = np.random.randint(len(train_inp), size=batch_size)
            batch_inp = inp_voc.to_matrix(train_inp[batch_ix]).to(device)
            batch_out = out_voc.to_matrix(train_out[batch_ix]).to(device)

            opt.zero_grad()
            loss_t = compute_loss(model, batch_inp, batch_out)
            loss_t.backward()
            opt.step()

            metrics['train_loss'].append((step, loss_t.item()))
            if step % 100 == 0:
                bleu = compute_bleu(model, dev_inp, dev_out)
                metrics['dev_bleu'].append((step, bleu))

                if bleu > best_bleu:
                    best_bleu = bleu
                    best_step = step
                    best_weights = model.state_dict()
                elif step - best_step >= n_iters_break:
                    break

                clear_output(wait=True)
                plt.figure(figsize=(12,4))
                for i, (name, history) in enumerate(sorted(metrics.items())):
                    plt.subplot(1, len(metrics), i + 1)
                    plt.title(name)
                    plt.plot(*zip(*history))
                    plt.grid()
                plt.show()

                print("Mean loss=%.3f" % np.mean(metrics['train_loss'][-10:], axis=0)[1], flush=True)
            
    model.load_state_dict(best_weights)
    return best_bleu

In [ ]:
# basic_model = BasicModel(inp_voc, out_voc).to(device)
# %time train_model(basic_model, batch_size=128, lr=1e-3*5, n_iters=5000)

In [ ]:
# torch.save(basic_model.state_dict(), "basic_model")
# basic_model.load_state_dict(torch.load("basic_model"))

In [ ]:
# def beam_research(model, inp, out, model_name, beam_min=1, beam_max=10):
#     beam_range = range(beam_min, beam_max + 1)
#     blue_scores = []

#     for beam_size in beam_range:
#         blue = compute_bleu(model, inp, out, beam_size=beam_size)
#         blue_scores.append(blue)

#     plt.figure(figsize=(7, 5))
#     plt.plot(beam_range, blue_scores)
#     plt.title('Beam search results for %s' % model_name)
#     plt.xticks(beam_range)
#     plt.xlabel('beam size')
#     plt.ylabel('blue score')
#     plt.show()

# %time beam_research(basic_model, dev_inp, dev_out, 'basic model')

In [ ]:
# with torch.no_grad():
#     for inp_line, trans_line in zip(dev_inp[::700], basic_model.translate_lines(dev_inp[::700], device, 2)[0]):
#         print('Inp: %s' % inp_line.replace('@@ ', ''))
#         print('Out: %s' % trans_line.replace('@@ ', ''))
#         print()

## Simple Encoder-decoder model gets bleu 17.5, but quality of translation is so bad

# Model with attention

### Attention layer

Here you will have to implement a layer that computes a simple additive attention:

Given encoder sequence $ h^e_0, h^e_1, h^e_2, ..., h^e_T$ and a single decoder state $h^d$,

* Compute logits with a 2-layer neural network
$$a_t = linear_{out}(tanh(linear_{e}(h^e_t) + linear_{d}(h_d)))$$
* Get probabilities from logits, 
$$ p_t = {{e ^ {a_t}} \over { \sum_\tau e^{a_\tau} }} $$

* Add up encoder states with probabilities to get __attention response__
$$ attn = \sum_t p_t \cdot h^e_t $$

You can learn more about attention layers in the lecture slides or [from this post](https://distill.pub/2016/augmented-rnns/).

## Seq2seq model with attention

You can now use the attention layer to build a network. The simplest way to implement attention is to use it in decoder phase. On every step, use __previous__ decoder state to obtain attention response. Then feed concat this response to the inputs of next attention layer.

The key implementation detail here is __model state__. Put simply, you can add any tensor into the list of `encode` outputs. You will then have access to them at each `decode` step. This may include:
* Last RNN hidden states (as in basic model)
* The whole sequence of encoder outputs (to attend to) and mask
* Attention probabilities (to visualize)

_There are alternative ways to wire attention into network and different kinds of attention. For example [this](https://arxiv.org/abs/1609.08144), [this](https://arxiv.org/abs/1706.03762) and [this](https://arxiv.org/abs/1808.03867) for ideas. And for image captioning/im2latex there's [visual attention](https://arxiv.org/abs/1502.03044)_

In [ ]:
from my_models import AttentiveModel

### Training attentive model

Please reuse the infrastructure you've built for the regular model. I hope you didn't hard-code anything :)

In [ ]:
# attentive_model = AttentiveModel(inp_voc, out_voc).to(device)
# %time train_model(attentive_model, n_iters=5000, lr=1e-3, batch_size=128)

In [ ]:
# torch.save(attentive_model.state_dict(), "att_model")
# attentive_model.load_state_dict(torch.load("att_model"))

In [ ]:
# %time beam_research(attentive_model, dev_inp, dev_out, 'attentive model')

In [ ]:
# with torch.no_grad():
#     for inp_line, trans_line in zip(dev_inp[::700], attentive_model.translate_lines(dev_inp[::700], device, 4)[0]):
#         print('Inp: %s' % inp_line.replace('@@ ', ''))
#         print('Out: %s' % trans_line.replace('@@ ', ''))
#         print()

## Attentive model gets bleu 26.2, it is significantly better then simple model and translation so better

In [ ]:
# %run my_models.py
scores = []
for bid in [True, False]:
    for rnn_type in ['LSTM', 'GRU', 'RNN']:
        model = AttentiveModel(inp_voc, out_voc, rnn_type=rnn_type, bid=bid).to(device)
        bleu = train_model(model, lr=1e-3*2, batch_size=10)
        scores.append(bleu)

## Transformer model

In [ ]:
# mname = "facebook/wmt19-ru-en"
# tokenizer = FSMTTokenizer.from_pretrained(mname)
# model = FSMTForConditionalGeneration.from_pretrained(mname).to('cuda:1')

In [ ]:
# def compute_blue_transformer(model, inp_lines, out_lines, bpe_sep='@@ '):
#     inp = [line.replace(bpe_sep, '') for line in inp_lines]
#     out = [line.replace(bpe_sep, '') for line in out_lines]
#     translations = []
#     for src in tqdm(inp):
#         input_ids = tokenizer.encode(src, return_tensors="pt").to('cuda:1')
#         outputs = model.generate(input_ids, num_beams=2)
#         decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         decoded = tokenize(decoded)
#         translations.append(decoded)
        
#     return corpus_bleu(
#         [[ref.split()] for ref in out],
#         [trans.split() for trans in translations],
#         smoothing_function=lambda precisions, **kw: [p + 1.0 / p.denominator for p in precisions]
#         ) * 100
    

# # for inp_line in dev_inp[::500]:
# #     inp = inp_line.replace('@@ ', '')
# #     input_ids = tokenizer.encode(inp, return_tensors="pt")
# #     outputs = model.generate(input_ids)
# #     decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
# #     decoded = tokenize(decoded)
# #     print(inp)
# #     print(decoded)
# #     print()
# compute_blue_transformer(model, dev_inp, dev_out)

In [ ]:
# input = "я не люблю 23 февраля и иду в бгу!"
# input_ids = tokenizer.encode(input, return_tensors="pt")
# outputs = model.generate(input_ids)
# decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(tokenize(decoded))

## Without Attention score was 16, and now is 21

### Visualizing model attention (2 points)

After training the attentive translation model, you can check it's sanity by visualizing its attention weights.

We provided you with a function that draws attention maps using [`Bokeh`](https://bokeh.pydata.org/en/latest/index.html). Once you managed to produce something better than random noise, please save at least 3 attention maps and __submit them to anytask__ alongside this notebook to get the max grade. Saving bokeh figures as __cell outputs is not enough!__ (TAs can't see saved bokeh figures in anytask). You can save bokeh images as screenshots or using this button:

![bokeh_panel](https://github.com/yandexdataschool/nlp_course/raw/2019/resources/bokeh_panel.png)

__Note:__ you're not locked into using bokeh. If you prefer a different visualization method, feel free to use that instead of bokeh.

In [ ]:
# import bokeh.plotting as pl
# import bokeh.models as bm
# from bokeh.io import output_notebook, show
# output_notebook()

# def draw_attention(inp_line, translation, probs):
#     """ An intentionally ambiguous function to visualize attention weights """
#     inp_tokens = inp_voc.tokenize(inp_line)
#     trans_tokens = out_voc.tokenize(translation)
#     probs = probs[:len(trans_tokens), :len(inp_tokens)]
    
#     fig = pl.figure(x_range=(0, len(inp_tokens)), y_range=(0, len(trans_tokens)),
#                     x_axis_type=None, y_axis_type=None, tools=[])
#     fig.image([probs[::-1]], 0, 0, len(inp_tokens), len(trans_tokens))

#     fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
#     fig.xaxis.ticker = np.arange(len(inp_tokens)) + 0.5
#     fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens)) + 0.5, inp_tokens))
#     fig.xaxis.major_label_orientation = 45

#     fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
#     fig.yaxis.ticker = np.arange(len(trans_tokens)) + 0.5
#     fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens)) + 0.5, trans_tokens[::-1]))

#     show(fig)

In [ ]:
# inp = dev_inp[::500]

# trans, states = model.translate_lines(inp)

# # select attention probs from model state (you may need to change this for your custom model)
# # attention_probs below must have shape [batch_size, translation_length, input_length], extracted from states
# # e.g. if attention probs are at the end of each state, use np.stack([state[-1] for state in states], axis=1)
# attention_probs = np.stack([state[-1].detach().cpu().numpy() for state in states], axis=1)

In [ ]:
# for i in range(5):
#     draw_attention(inp[i], trans[i], attention_probs[i])

# Из рисунков видно, что есть Attention, которые смотрят на предыдущие токены. Есть, которые на следующие. Есть, которые еще на что-то интересное. PDF с ними сдам в lms.

__Note 1:__ If the attention maps are not iterpretable, try starting encoder from zeros (instead of dec_start), forcing model to use attention.

__Note 2:__ If you're studying this course as a YSDA student, please submit __attention screenshots__ alongside your notebook.

## Goind deeper (2++ points each)

We want you to find the best model for the task. Use everything you know.

* different recurrent units: rnn/gru/lstm; deeper architectures
* bidirectional encoder, different attention methods for decoder (additive, dot-product, multi-head)
* word dropout, training schedules, anything you can imagine
* replace greedy inference with beam search

For a better grasp of seq2seq We recommend you to conduct at least one experiment from one of the bullet-points or your alternative ideas. As usual, describe what you tried and what results you obtained in a short report.

# А теперь попробуем bidirectional GRU с нашим первым Attention


# Было blue 20.9, а щас целых 22.32. То есть Bidirectional GRU лучше обычного GRU. Значит с большой вероятностью можно утверждать, что Bid. LSTM и Bid. RNN (обычная RNN) будут лучше своих не Bid. версий. Сначала проверим Bid. RNN

# RNN сработала примерно также как и GRU, странно, ведь GRU - модификация RNN, а сейчас проверим LSTM

# Стало еще хуже что-то. Как так? Ведь LSTM лучше RNN должно быть - вот и я не знаю. 

# Давайте к нашему лучшему варианту (Bid. GRU - дает 22.32 bleu) чуть подберем константы hid_size, emb_size и т.д.

In [ ]:
# class AttentiveModel(BasicModel):
#     def __init__(self, name, inp_voc, out_voc,
#                  emb_size=300, hid_size=256, attn_size=256):
#         """ Translation model that uses attention. See instructions above. """
#         nn.Module.__init__(self)  # initialize base class to track sub-layers, trainable variables, etc.
#         self.inp_voc, self.out_voc = inp_voc, out_voc
#         self.hid_size = hid_size
        
#         self.emb_inp = nn.Embedding(len(inp_voc), emb_size)
#         self.emb_out = nn.Embedding(len(out_voc), emb_size)
#         self.enc0 = nn.GRU(emb_size, hid_size, batch_first=True, bidirectional=True)

#         self.dec_start = nn.Linear(hid_size*2, hid_size)
#         self.dec0 = nn.GRUCell(emb_size+hid_size*2, hid_size)
#         self.logits = nn.Linear(hid_size, len(out_voc))
        
#         self.attn = AttentionLayer('kek', hid_size*2, hid_size, hid_size)

#     def encode(self, inp, **flags):
#         """
#         Takes symbolic input sequence, computes initial state
#         :param inp: matrix of input tokens [batch, time]
#         :return: a list of initial decoder state tensors
#         """
        
#         inp_emb = self.emb_inp(inp)
#         batch_size = inp.shape[0]
        
#         # enc_seq = inp_emb
#         enc_seq, _ = self.enc0(inp_emb)
#         # print(enc_seq.shape)
#         # enc_seq: [batch, time, hid_size], last_state: [batch, hid_size]
        
#         # note: last_state is not _actually_ last because of padding, let's find the real last_state
#         lengths = (inp != self.inp_voc.eos_ix).to(torch.int64).sum(dim=1).clamp_max(inp.shape[1] - 1)
#         last_state = enc_seq[torch.arange(len(enc_seq)), lengths]
#         # ^-- shape: [batch_size, hid_size]
        
#         dec_start = self.dec_start(last_state)        
        
#         enc_mask = self.out_voc.compute_mask(inp)
        
#         # apply attention layer from initial decoder hidden state
#         first_attn = self.attn(enc_seq, dec_start, enc_mask)[1]
        
#         # Build first state: include
#         # * initial states for decoder recurrent layers
#         # * encoder sequence and encoder attn mask (for attention)
#         # * make sure that last state item is attention probabilities tensor
        
#         first_state = [dec_start, enc_seq, enc_mask, first_attn]
#         return first_state
    
   
#     def decode_step(self, prev_state, prev_tokens, **flags):
#         """
#         Takes previous decoder state and tokens, returns new state and logits for next tokens
#         :param prev_state: a list of previous decoder state tensors
#         :param prev_tokens: previous output tokens, an int vector of [batch_size]
#         :return: a list of next decoder state tensors, a tensor of logits [batch, n_tokens]
#         """
        
#         prev_gru0_state, enc_seq, enc_mask, _ = prev_state
        
#         attn, attn_prob = self.attn(enc_seq, prev_gru0_state, enc_mask)
        
#         x = self.emb_out(prev_tokens)
#         assert len(x.shape) == 2 and len(attn.shape) == 2
#         x = torch.cat([attn, x], dim=-1)
#         x = self.dec0(x, prev_gru0_state)
        
#         new_dec_state = [x, enc_seq, enc_mask, attn_prob]
#         output_logits = self.logits(x)
        
#         return new_dec_state, output_logits

# metrics = {'train_loss': [], 'dev_bleu': [] }

# model = AttentiveModel('имя (name)', inp_voc, out_voc).to('cuda:1')
# opt = torch.optim.Adam(model.parameters(), lr=1e-3)
# batch_size = 64

# for _ in trange(4200):
#     step = len(metrics['train_loss']) + 1
#     batch_ix = np.random.randint(len(train_inp), size=batch_size)
#     batch_inp = inp_voc.to_matrix(train_inp[batch_ix]).to('cuda:1')
#     batch_out = out_voc.to_matrix(train_out[batch_ix]).to('cuda:1')
    
#     opt.zero_grad()
#     loss_t = compute_loss(model, batch_inp, batch_out)
#     loss_t.backward()
#     opt.step()
    
#     metrics['train_loss'].append((step, loss_t.item()))
    
#     if step % 100 == 0:
#         metrics['dev_bleu'].append((step, compute_bleu(model, dev_inp, dev_out)))
        
#         clear_output(True)
#         plt.figure(figsize=(12,4))
#         for i, (name, history) in enumerate(sorted(metrics.items())):
#             plt.subplot(1, len(metrics), i + 1)
#             plt.title(name)
#             plt.plot(*zip(*history))
#             plt.grid()
#         plt.show()
#         print("Mean loss=%.3f" % np.mean(metrics['train_loss'][-10:], axis=0)[1], flush=True)
        
# for inp_line, trans_line in zip(dev_inp[::500], model.translate_lines(dev_inp[::500])[0]):
#     print(inp_line)
#     print(trans_line)
#     print()

# compute_bleu(model, dev_inp, dev_out)

# 24.8 - уже совсем много, думаю на этом можно закончить